<a href="https://colab.research.google.com/github/Linho1150/Automatic_arasaac/blob/main/kafka-producer-consumer-example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kafka-python

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 246 kB 4.9 MB/s 


In [ ]:
from datetime import datetime
import threading, time

from kafka import KafkaAdminClient, KafkaConsumer, KafkaProducer
from kafka.admin import NewTopic

#카프카 프로듀서 운영 객체
class Producer():
    bootstrap_servers='43.201.95.87:9092'
    def sendMessage(self,topic_name:str, message:str):
        producer = KafkaProducer(bootstrap_servers=Producer.bootstrap_servers)
        producer.send(topic_name, message.encode('utf-8'))
        producer.close()

#카프카 스트리밍 프로듀서 운영 객체
class StreamingProducer():
    bootstrap_servers='43.201.95.87:9092'
    def sendMessage(self, message:str):
        producer = KafkaProducer(bootstrap_servers=Producer.bootstrap_servers)
        producer.send('streams-plaintext-input', message.encode('utf-8'))
        producer.close()

#카프카 컨슈머 객체
class Consumer():
    bootstrap_servers = '43.201.95.87:9092'
    def getMessage(self, topic_name:str):
      consumer = KafkaConsumer(bootstrap_servers=Consumer.bootstrap_servers,
                                auto_offset_reset='earliest',
                                consumer_timeout_ms=1000)
      consumer.subscribe([topic_name])
      print(consumer)
      for message in consumer:
        print("Topic: ",message.topic,"Message: ",message.value)
      consumer.close()

#카프카 토픽 운영 객체
class KafkaTopicAdministrator:
    kafkaAdmin = KafkaAdminClient(bootstrap_servers='43.201.95.87:9092')

    def setKafkaTopic(self, topic_name: str): # Set topic
        if not self.isKafkaTopic(topic_name):
            topic = NewTopic(name=topic_name,
                             num_partitions=1,
                             replication_factor=1)
            return KafkaTopicAdministrator.kafkaAdmin.create_topics([topic])
        raise Exception("already exist")

    def getKafkaTopicList(self) -> list: # Get topic list
        return KafkaTopicAdministrator.kafkaAdmin.list_topics()

    def deleteKafkaTopic(self, topic_name: str) -> list: # Delete topic
        if self.isKafkaTopic(topic_name):
            return KafkaTopicAdministrator.kafkaAdmin.delete_topics([topic_name])
        raise Exception("no topic")

    def isKafkaTopic(self, topic_name: str): # Check if topic exists
        if topic_name in KafkaTopicAdministrator.kafkaAdmin.list_topics():
            return True
        return False


if __name__ == "__main__":
    # Kafka Topic Manager
    kafkaAdmin = KafkaTopicAdministrator() # Kafka 관리자 객체 생성
    kafkaAdmin.setKafkaTopic("my-kafka-example") # Kafka 토픽 생성
    kafkaAdmin.deleteKafkaTopic("my-kafka-example") # Kafka 토픽 제거
    print(kafkaAdmin.getKafkaTopicList()) #Kafka 토픽 리스트

    # Kafka Producer & Consumer
    producer=Producer()
    producer.sendMessage("my-kafka","Kafka")
    consumer=Consumer()
    consumer.getMessage("my-kafka")

    # Kafka Streaming Producer
    producer = KafkaProducer(bootstrap_servers='43.201.95.87:9092')
    streamingProducer=StreamingProducer()
    while True:
      # Infinity loop (Can't break 🤪)
      inputTxt=input().encode('utf-8')
      streamingProducer.sendMessage(inputTxt)

